<a href="https://colab.research.google.com/github/bs3537/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/OK_Copy_of_Bhav_CNNs_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

IMPORT LIBRARIES

In [0]:
import os
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import tensorflow
import pandas as pd
import itertools
from tensorflow import keras
 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model, Sequential # This is the functional API
from tensorflow.keras.optimizers import RMSprop, Adam, Nadam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.preprocessing import image

from tensorflow.keras import applications 
from keras.utils.np_utils import to_categorical 

from sklearn import metrics
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
%matplotlib inline
import math 
import sys
import cv2
 

In [0]:
#CREATE MODEL

resnet = ResNet50(input_shape=(224,224, 3), include_top=False, weights='imagenet')
for layer in resnet.layers:
    layer.trainable = False

x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x) # 1 for binary classification (otherwise no. of classes). for binary classification, use sigmoid, else softmax
model = Model(resnet.input, predictions)

In [0]:
#Compile the model

base_learning_rate = 0.001
model.compile(optimizer=tensorflow.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tensorflow.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [39]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [0]:

 
# loading up our datasets

TRAINING_DIR = "/content/drive/My Drive/Lambda DS_Unit 4 Deep Learning/CNNs/data/train"

VALIDATION_DIR = "/content/drive/My Drive/Lambda DS_Unit 4 Deep Learning/CNNs/data/validation"
 

#ImageDataGenerator = tensorflow.keras.preprocessing.image.ImageDataGenerator

In [0]:
# Load images into numpy arrays/ create y for labels

#TRAIN DATA 

FILEPATH = "/content/drive/My Drive/Lambda DS_Unit 4 Deep Learning/CNNs/data/train/"
CATEGORIES = ['forest', 'mountain']
training_data = []
IMG_SIZE = 224

def create_training_data():
  for category in CATEGORIES:
    path = os.path.join(FILEPATH, category) 
    class_num = CATEGORIES.index(category)
    print(path, class_num)
    for img in os.listdir(path):
      print(img)
      try:
        #if not img.startswith(\".jpg\"):
        img_array = cv2.imread(os.path.join(path, img))
        new_img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training_data.append([new_img_array, class_num])
      except Exception as e:
        pass
  X = []
  y = []

  for features, label in training_data:
    X.append(features)
    y.append(label)

    X = np.array(X)
    y = to_categorical(np.array(y))
    print('------- Shape of X, y data -------')
    print(X.shape, y.shape)
    return X, y


In [34]:
X_train, y_train = create_training_data()

/content/drive/My Drive/Lambda DS_Unit 4 Deep Learning/CNNs/data/train/forest 0
for79.jpg
text111.jpg
land102.jpg
text107.jpg
land159.jpg
land872.jpg
natu325.jpg
natu696.jpg
nat982.jpg
nat130.jpg
natu361.jpg
nat417.jpg
nat851.jpg
text90.jpg
text103.jpg
for114.jpg
land314.jpg
text48.jpg
land760.jpg
natu611.jpg
land849.jpg
for3.jpg
nat623.jpg
natu723.jpg
for86.jpg
for84.jpg
for78.jpg
nat203.jpg
nat354.jpg
nat763.jpg
natu871.jpg
nat338.jpg
natu934.jpg
natu430.jpg
nat442.jpg
text124.jpg
nat146.jpg
for149.jpg
natu863.jpg
nat228.jpg
nat1127.jpg
text40.jpg
land107.jpg
land727.jpg
nat835.jpg
nat1131.jpg
for102.jpg
text44.jpg
natu163.jpg
nat470.jpg
natu847.jpg
nat223.jpg
land222.jpg
for63.jpg
text26.jpg
nat400.jpg
land854.jpg
land861.jpg
for65.jpg
natu164.jpg
natu29.jpg
nat359.jpg
land81.jpg
natu848.jpg
natu869.jpg
nat347.jpg
tell51.jpg
nat205.jpg
for127.jpg
land869.jpg
natu424.jpg
land810.jpg
nat619.jpg
text29.jpg
land726.jpg
land765.jpg
natu2.jpg
for17.jpg
nat360.jpg
nat325.jpg
for136.jpg
for

In [0]:
# Load images into numpy arrays/ create y for labels\n",

#VAL. DATA 

FILEPATH = "/content/drive/My Drive/Lambda DS_Unit 4 Deep Learning/CNNs/data/validation/"
CATEGORIES = ['forest', 'mountain']
validation_data = []
IMG_SIZE = 224

def create_validation_data():
  for category in CATEGORIES:
    path = os.path.join(FILEPATH, category) 
    class_num = CATEGORIES.index(category)
    print(path, class_num)
    for img in os.listdir(path):
      print(img)
      try:
        #if not img.startswith(\".jpg\"):
        img_array = cv2.imread(os.path.join(path, img))
        new_img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training_data.append([new_img_array, class_num])
      except Exception as e:
        pass
  X = []
  y = []

  for features, label in training_data:
    X.append(features)
    y.append(label)

    X = np.array(X)
    y = to_categorical(np.array(y))
    print('------- Shape of X, y data -------')
    print(X.shape, y.shape)
    return X, y

In [36]:
X_val, y_val = create_validation_data()

/content/drive/My Drive/Lambda DS_Unit 4 Deep Learning/CNNs/data/validation/forest 0
cdmc458.jpg
cdmc331.jpg
cdmc333.jpg
cdmc280.jpg
cdmc385.jpg
cdmc319.jpg
n18075.jpg
cdmc507.jpg
cdmc292.jpg
cdmc556.jpg
natc52.jpg
cdmc352.jpg
bost103.jpg
cdmc363.jpg
bost100.jpg
cdmc277.jpg
cdmc306.jpg
cdmc313.jpg
cdmc547.jpg
cdmc533.jpg
cdmc278.jpg
cdmc290.jpg
cdmc317.jpg
cdmc430.jpg
cdmc283.jpg
bost98.jpg
cdmc415.jpg
bost190.jpg
nat1266.jpg
cdmc451.jpg
cdmc284.jpg
cdmc318.jpg
cdmc494.jpg
cdmc414.jpg
cdmc282.jpg
cdmc101.jpg
cdmc388.jpg
cdmc281.jpg
cdmc344.jpg
cdmc12.jpg
natc13.jpg
natc12.jpg
cdmc375.jpg
cdmc351.jpg
cdmc315.jpg
cdmc348.jpg
cdmc271.jpg
natc4.jpg
cdmc368.jpg
cdmc359.jpg
cdmc358.jpg
cdmc377.jpg
cdmc338.jpg
cdmc293.jpg
bost101.jpg
cdmc291.jpg
bost102.jpg
cdmc562.jpg
natc37.jpg
natc49.jpg
/content/drive/My Drive/Lambda DS_Unit 4 Deep Learning/CNNs/data/validation/mountain 1
n18019.jpg
n213096.jpg
n213041.jpg
n44001.jpg
n603048.jpg
n219015.jpg
n347049.jpg
n18058.jpg
n213009.jpg
n738013.jpg
n

TRAIN AND VALIDATE ON DATA

In [41]:
epochs = 10

batch_size=10

stop = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.001, patience=3, verbose=1)
filepath="best_resnet_model_scenes_assignment.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_freq='epoch')

history = model.fit(
          X_train, y_train,
          epochs=epochs,
          validation_data=(X_val, y_val),
          verbose=1,
          callbacks=[stop, checkpoint],
          batch_size=batch_size)


Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.3133 - accuracy: 1.0000
Epoch 00001: val_loss improved from inf to 0.31326, saving model to best_resnet_model_scenes_assignment.hdf5
1/1 [==============================] - 1s 575ms/step - loss: 0.3133 - accuracy: 1.0000 - val_loss: 0.3133 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.3133 - accuracy: 1.0000
Epoch 00002: val_loss did not improve from 0.31326
1/1 [==============================] - 0s 44ms/step - loss: 0.3133 - accuracy: 1.0000 - val_loss: 0.3133 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.3133 - accuracy: 1.0000
Epoch 00003: val_loss did not improve from 0.31326
1/1 [==============================] - 0s 43ms/step - loss: 0.3133 - accuracy: 1.0000 - val_loss: 0.3133 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.3133 - accuracy: 1.0000
Epoch 00004: val_loss did not improv

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [0]:
# Setup Architecture
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [0]:
# Compile Model

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [53]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 52, 52, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 173056)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)               

In [54]:
# Fit Model
epochs = 10

batch_size=10

stop = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.001, patience=3, verbose=1)
filepath="best_resnet_model_scenes_assignment2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_freq='epoch')

history = model.fit(
          X_train, y_train,
          epochs=epochs,
          validation_data=(X_val, y_val),
          verbose=1,
          callbacks=[stop, checkpoint],
          batch_size=batch_size)

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 1.1073e-12 - accuracy: 1.0000
Epoch 00001: val_loss improved from inf to 0.00000, saving model to best_resnet_model_scenes_assignment2.hdf5
1/1 [==============================] - 0s 360ms/step - loss: 1.1073e-12 - accuracy: 1.0000 - val_loss: 1.1073e-12 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 1.1073e-12 - accuracy: 1.0000
Epoch 00002: val_loss did not improve from 0.00000
1/1 [==============================] - 0s 37ms/step - loss: 1.1073e-12 - accuracy: 1.0000 - val_loss: 1.1073e-12 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 1.1073e-12 - accuracy: 1.0000
Epoch 00003: val_loss did not improve from 0.00000
1/1 [==============================] - 0s 34ms/step - loss: 1.1073e-12 - accuracy: 1.0000 - val_loss: 1.1073e-12 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 1.1073e-12 - accuracy: 1.

# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [0]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language